# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Daniel Faria

### Grupo 03
<dl>
    <dt>18 horas (33.3%)</dt>
    <dd>ist199309 Rafael Girão</dd>
    <dt>18 horas (33.3%)</dt>
    <dd>ist102082 Simão Sanguinho</dd>
    <dt>18 horas (33.3%)</dt>
    <dd>ist103252 José Pereira</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db
%config SqlMagic.displaylimit = None

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;
DROP VIEW IF EXISTS product_sales CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql
-- populate.sql

INSERT INTO customer
VALUES
(-1, 'Deleted User', 'deleted@localhost.localdomain', '000000000', 'null'), --Dummy user to hold a deleted customer's leftovers.
(13, 'João Faria', 'joao.faria@gmail.com', '913789392', 'Rua Camões 123, 4800-005 Guimarães'),
(2, 'Jorge Nunes', 'jorge.n@hotmail.com', '987712312', 'Rua Amarela 123, 8700-312 Olhão'),
(35, 'Ana Lampreia', 'ana.lamp@gmail.com', '91203456', 'Rua dos Cravos 12, 1049-001 Lisboa');

INSERT INTO orders
VALUES
(101, 13, '2022-01-01'),
(102, 2, '2023-01-02'),
(103, 35, '2022-05-03'),
(104, 2, '2023-05-03'),
(105, 35, '2022-01-03'),
(106, 35, '2022-02-03'),
(107, 35, '2022-03-03'),
(108, 2, '2022-04-03'),
(109, 35, '2022-05-03'),
(110, 35, '2022-06-03'),
(111, 2, '2023-08-18'),
(112, 35, '2022-07-03'),
(113, 35, '2022-08-03'),
(114, 13, '2022-09-03'),
(115, 13, '2022-10-03'),
(116, 13, '2022-11-03'),
(117, 2, '2022-12-03'),
(118, 35, '2022-10-03');

INSERT INTO pay 
VALUES
(101, 13),
(102, 2),
(104, 2);

INSERT INTO employee
VALUES
('12345678910', '243678525', '2000-01-01', 'Sofia Barbosa'),
('10987654321', '125637263', '1985-05-10', 'Diogo Silva');

INSERT INTO process
VALUES
('12345678910', 101),
('10987654321', 102),
('12345678910', 103),
('10987654321', 104),
('12345678910', 105),
('12345678910', 106),
('12345678910', 107),
('12345678910', 108),
('12345678910', 109),
('12345678910', 110),
('12345678910', 111),
('12345678910', 112),
('12345678910', 113),
('12345678910', 114),
('12345678910', 115),
('12345678910', 116),
('12345678910', 117),
('10987654321', 118);

INSERT INTO department
VALUES
('Sales'),
('Accounting');

INSERT INTO workplace 
VALUES
('Rua 25 de Abril 15, 3100-440 Pombal', 37.1234, -122.5678),
('Avenida das Pombas 2, 2844-001 Seixal', 38.9876, -121.3456);


INSERT INTO works 
VALUES
('12345678910', 'Sales', 'Rua 25 de Abril 15, 3100-440 Pombal'),
('10987654321', 'Accounting', 'Avenida das Pombas 2, 2844-001 Seixal');

INSERT INTO office 
VALUES
('Rua 25 de Abril 15, 3100-440 Pombal');

INSERT INTO warehouse 
VALUES
('Avenida das Pombas 2, 2844-001 Seixal');

INSERT INTO product 
VALUES
('1001A', 'Caneta', 'Caneta preta', 1, '3030303030303'),
('1002B', 'Móvel', 'Móvel branco de madeira', 35, '4444444444444'),
('1003C', 'Cama', 'Cama Queen-Size', 299, '5555555555555');


INSERT INTO supplier
VALUES
('983747222', 'Joana Pereira', 'Rua Verde 230, 2770-022 Paço de Arcos', '1001A', '2023-02-02'),
('124455551', 'Jacinto Torres', 'Rua Azul 123, 1500-173 Amadora', '1002B', '2023-01-01'),
('525258733', 'Miguel Leal', 'Rua Camoes 17, 7780-217 Castro Verde', '1003C', '2023-03-03');

INSERT INTO contains
VALUES
(101, '1001A', 2),
(101, '1002B', 6),
(102, '1001A', 2),
(103, '1003C', 1),
(104, '1003C', 6),
(105, '1003C', 1),
(106, '1003C', 1),
(107, '1003C', 1),
(108, '1003C', 1),
(109, '1003C', 1),
(110, '1003C', 1),
(111, '1003C', 1),
(112, '1003C', 1),
(113, '1003C', 1),
(114, '1003C', 1),
(115, '1003C', 1),
(116, '1003C', 1),
(117, '1003C', 1),
(118, '1003C', 1);

INSERT INTO delivery
VALUES
('Avenida das Pombas 2, 2844-001 Seixal', 983747222),
('Avenida das Pombas 2, 2844-001 Seixal', 124455551),
('Avenida das Pombas 2, 2844-001 Seixal', 525258733);


## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
ALTER TABLE employee 
   ADD CONSTRAINT check_18 CHECK (EXTRACT(YEAR FROM AGE(bdate)) > 18);

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION check_workplace_type() RETURNS TRIGGER AS $$
DECLARE
  office_count INTEGER;
  warehouse_count INTEGER;
BEGIN
  SELECT COUNT(*) INTO office_count FROM office WHERE address = NEW.address;
  SELECT COUNT(*) INTO warehouse_count FROM warehouse WHERE address = NEW.address;

  IF (office_count > 0 AND warehouse_count > 0) OR (office_count = 0 AND warehouse_count = 0)THEN
    RAISE EXCEPTION 'O local de trabalho deve ser um "Office" ou "Warehouse", mas não pode ser ambos ou nenhum deles.';
  END IF;

  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_workplace_type_trigger
AFTER INSERT OR UPDATE ON workplace 
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_workplace_type();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql 

CREATE OR REPLACE FUNCTION check_order_contains() RETURNS TRIGGER AS $$
DECLARE
    order_count INTEGER;
BEGIN
  SELECT COUNT(*) INTO order_count FROM contains WHERE order_no = NEW.order_no;

  IF (order_count = 0) THEN
    RAISE EXCEPTION 'A ordem deve figurar obrigatoriamente em "Contains".';
  END IF;

  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_order_contains_trigger
AFTER INSERT OR UPDATE ON orders
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_order_contains();

## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

SELECT cust_no, name 
FROM customer
    JOIN pay USING (cust_no)
GROUP BY cust_no
HAVING COUNT(*) >= ALL(
    SELECT COUNT(*) 
    FROM customer
    JOIN pay USING (cust_no)
    GROUP BY cust_no);

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql
SELECT name FROM employee
WHERE NOT EXISTS (
    SELECT date
    FROM orders
    WHERE EXTRACT(YEAR FROM date) = 2022
    EXCEPT
    SELECT date
    FROM orders
    INNER JOIN process
    ON process.order_no = orders.order_no
  WHERE process.ssn = employee.ssn
);

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql
SELECT EXTRACT(MONTH FROM date) AS month, COUNT(*) AS num_orders
FROM orders
WHERE EXTRACT(YEAR FROM date) = 2022
AND NOT EXISTS (
  SELECT *
  FROM pay
  WHERE pay.order_no = orders.order_no
)
GROUP BY month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql
CREATE VIEW product_sales AS
SELECT c.SKU, o.order_no, c.qty, (c.qty * p.price) AS total_price, EXTRACT(YEAR FROM o.date) AS year,
       EXTRACT(MONTH FROM o.date) AS month, EXTRACT(DAY FROM o.date) AS day_of_month,
       EXTRACT(DOW FROM o.date) AS day_of_week, 
       SUBSTRING(cust.address, LENGTH(cust.address) - POSITION(',' IN REVERSE(cust.address)) + 2) AS city
FROM contains c
JOIN orders o ON c.order_no = o.order_no
JOIN pay py ON o.order_no = py.order_no
JOIN product p ON c.SKU = p.SKU
JOIN customer cust ON o.cust_no = cust.cust_no;

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

### Introdução
A web app desenvolvida apresenta um Sistema de Gestão de Encomendas de Clientes - BDelivery - implementado em Python, que auxilia na gestão eficiente de encomendas, produtos, clientes e fornecedores.

### Visão Geral do Sistema
O Sistema BDelivery é uma aplcicação web desenvolvida em Python com o framework Flask. Utiliza uma base de dados PostgreSQL (integrada na aplicação com a biblioteca psycopg3)  para armazenar informações de clientes, encomendas, produtos e fornecedores. A apresentação dos dados utiliza templates HTML, com CSS do framework Tailwind.

### Funcionalidades
- Gestão de Clientes: Listagem, inserção, edição e remoção de clientes.
- Gestão de Encomendas: Listagem, inserção e pagamentos¹ de encomendas.
- Gestão de Produtos: Listagem, inserção, edição e remoção de produtos disponíveis.
- Gestão de Fornecedores: Listagem, inserção e remoção de fornecedores.
- Paginação: Exibição limitada de itens por página. ²
$$
\begin {align*}
& \text{¹ - A funcionalidade de pagamento de encomendas apenas está disponível a partir do cliente que as realizou.}\newline
& \text{² - São exibidos até 15 itens por página.}
\end{align*}
$$


### Componentes do Sistema
- Aplicação Web Flask: Construída com Flask para lidar com pedidos HTTP.
- Base de Dados PostgreSQL: Armazena e gerencia informações do sistema.
- Templates HTML: Apresentação de páginas web³ 
- Função de Paginação: Melhora a navegação ao exibir números de página apropriados.
$$\begin {align*}
& \text{³ - Foi criado, para cada categoria uma diretoria que contém um template para cada um  das funcionalidades de inserção, listagem e edição que lhe pertencem. }
\end {align*}
$$


### Segurança
Existem dois grandes vetores de possível violação da segurança da aplicação web: injeção de dados arbitrários nas consultas SQL e injeção arbitrária no frontend ( por via dos placeholders contidos nos templates de HTML). O caso de injeção SQL é salvaguardado pela sanitização automática dos placeholders das consultas pela biblioteca psycopg3. O mesmo acontece no caso dos placeholders das templates de Jinja: esta biblioteca, por omissão, sanitiza todos os placeholders para que não ocorra injeção de código HTML arbirtário por estas.


### Conclusão
O Sistema de Gestão de Encomendas de Clientes oferece uma interface web amigável para a gestão de encomendas e informações de produtos. Com recursos como gestão de clientes, encomendas, fornecedores e produtos, o sistema visa otimizar os processos de negócio.






## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
SELECT
    SKU,
    city,
    month,
    day_of_month,
    day_of_week,
    SUM(qty) AS total_quantity,
    SUM(total_price) AS total_sales
FROM product_sales
WHERE year = 2022
GROUP BY GROUPING SETS((SKU), (SKU, city), (SKU, month), (SKU, day_of_month),(SKU, day_of_week))
ORDER BY SKU, city, month, day_of_month, day_of_week;


2. O valor médio diário das vendas em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
DROP TABLE IF EXISTS dateD;
CREATE TABLE dateD (
  date DATE,
  month INT,
  day_of_week INT,
  day_of_month INT
);

INSERT INTO dateD (date, month, day_of_week, day_of_month)
SELECT
  date,
  EXTRACT(MONTH FROM date) AS month,
  EXTRACT(DOW FROM date) AS day_of_week, 
  EXTRACT(DAY FROM date) AS day_of_month
FROM generate_series('2022-01-01'::date, '2022-12-31'::date, '1 day'::interval) AS date;

WITH tmp AS (SELECT d.date as dateS, d.month, d.day_of_week, SUM(COALESCE(p.total_price, 0)) as TOTPRICE
    FROM dateD d
    LEFT JOIN product_sales p ON (p.month = d.month AND p.day_of_month = d.day_of_month AND EXTRACT(YEAR FROM d.date) = p.year AND p.year = 2022)
    GROUP BY (d.date, d.month, d.day_of_week)
    ORDER BY d.date)

SELECT month, day_of_week, ROUND(AVG(totprice),2) as average
  FROM tmp
  GROUP BY GROUPING SETS ((), (month), (day_of_week))
  ORDER BY month, day_of_week;

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

In [ ]:
%%sql

CREATE INDEX idx_order_date ON orders(date);
CREATE INDEX product_price ON product(price);



- justificação:

    No primeiro JOIN, como order_no é uma chave primária de order, pelo que não faz sentido criar um índice.

    No segundo JOIN como SKU integra a chave primária, pelo que não faz sentido criar um índice.

    idx_order_date é um indice que orderna a tabela orders pela coluna date. O índice é criado na tabela orders, sobre o atributo date, que é o atributo que está a ser comparado na condição de seleção. Escolheu-se um indice do tipo btree, pois a procura é feita pelo range de datas (datas com o ano 2023).

    idx_product_price é um indice que orderna a tabela product pela coluna price. O índice é criado na tabela product, sobre o atributo price, que é o atributo que está a ser comparado na condição de seleção. Escolheu-se um indice do tipo btree, pois a procura é feita pelo range de preços (preços maiores que 50).



    


### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

In [ ]:
%%sql

CREATE INDEX idx_product_name ON product(name);
CREATE INDEX idx_contains_sku ON contains USING HASH(SKU);

- justificação:
    
    O GROUP BY order_no beneficia de um indice em contains, no entanto o PostgresSQL já cria um indice para order_no pelo facto de fazer parte da chave primaria. Como o planer já usa esse indice não faz sentido criar outro.

    Da mesma forma, como o SKU é chave primária de product, não é necessário criar um indice para esta variável, no âmbito do JOIN product USING(SKU).

    idx_product_name é um indice que ordena a tabela products pela coluna name. Visto que a variável name tem de ter um prefixo "A", como indicado na comparação WHERE name LIKE 'A%', criou-se um índice do tipo btree sobre a variável name, na tabela product. Foi escolhido o tipo btree para o índice visto que este tipo é capaz de agilizar operações de pattern matching.

    idx_contains_sku é um indice que ordena a tabela contains pela coluna SKU. O objetivo é melhorar eficiencia do JOIN entre contains e product. O contains está organizado fisicamente pela chave primária (order_no, sku) enquanto products está por (sku). Ao criar um indice sobre SKU em contains, o PostgresSQL pode usar o indice para aceder a contains de forma mais eficiente. Usou-se um indice do tipo hash pois procura-se o valor exato do SKU e não um range de valores.